
From Kaggle: 
"MNIST ("Modified National Institute of Standards and Technology") is the de facto “hello world” dataset of computer vision. Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. As new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike."

[Read more.](https://www.kaggle.com/c/digit-recognizer)


<a title="By Josef Steppan [CC BY-SA 4.0 (https://creativecommons.org/licenses/by-sa/4.0)], from Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:MnistExamples.png"><img width="512" alt="MnistExamples" src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png"/></a>

This code is adopted from the pytorch examples repository. 
It is licensed under BSD 3-Clause "New" or "Revised" License.
Source: https://github.com/pytorch/examples/
LICENSE: https://github.com/pytorch/examples/blob/master/LICENSE

![](https://github.com/rpi-techfundamentals/fall2018-materials/blob/master/10-deep-learning/mnist-comparison.png?raw=1)
Table from [Wikipedia](https://en.wikipedia.org/wiki/MNIST_database)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Pytorch Advantages vs Tensorflow
- Pytorch Enables dynamic computational graphs (which change be changed) while Tensorflow is static. 
- Tensorflow enables easier deployment. 

In [ ]:
#Import Libraries


from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sklearn.metrics as sk
import PIL
import matplotlib.pyplot as plt
from skimage.util import random_noise
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
from PIL import Image, ImageOps, ImageEnhance



In [ ]:
args={}
kwargs={}
args['batch_size']=1000
args['test_batch_size']=1000
args['epochs']=20  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.01 #Learning rate is how fast it will decend. 
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args['seed']=1 #random seed
args['log_interval']=10
args['cuda']=True

to_tensor = transforms.ToTensor()
normalize = transforms.Normalize([0.5] * 3, [0.5] * 3)


In [ ]:
#load the data
train_dataset=datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
train_data,val_data= torch.utils.data.random_split(train_dataset, [50000, 10000])
train_dataset1=datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([transforms.ToTensor(),transforms.Lambda(lambda x: x.repeat(3, 1, 1) ),transforms.ToPILImage()]))
train_data1,val_data1= torch.utils.data.random_split(train_dataset1, [50000, 10000])
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['test_batch_size'], shuffle=False, **kwargs)
fmnist_dataset=datasets.FashionMNIST('./data', train=True, download=True, transform=transforms.ToTensor())
fmnist_loader=torch.utils.data.DataLoader(fmnist_dataset,batch_size=2000,shuffle=False,**kwargs)
fake_dataset=datasets.KMNIST('./data',train=False, download=True, transform = transforms.ToTensor())
fake_loader=torch.utils.data.DataLoader(fake_dataset,batch_size=2000,shuffle=False,**kwargs)
train_loader = torch.utils.data.DataLoader(train_data,batch_size=args['batch_size'], shuffle=True, **kwargs)
ood_num_examples= len(train_data)//5
mnist_c_d1=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/brightness/test_images.npy'))
mnist_c_d2=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/canny_edges/test_images.npy'))
mnist_c_d3=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/dotted_line/test_images.npy'))
mnist_c_d4=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/fog/test_images.npy'))
mnist_c_d5=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/glass_blur/test_images.npy'))
mnist_c_d6=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/identity/test_images.npy'))
mnist_c_d7=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/impulse_noise/test_images.npy'))
mnist_c_d8=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/motion_blur/test_images.npy'))
mnist_c_d9=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/rotate/test_images.npy'))
mnist_c_d10=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/scale/test_images.npy'))
mnist_c_d11=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/shear/test_images.npy'))
mnist_c_d12=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/shot_noise/test_images.npy'))
mnist_c_d13=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/spatter/test_images.npy'))
mnist_c_d14=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/stripe/test_images.npy'))
mnist_c_d15=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/translate/test_images.npy'))
mnist_c_d16=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/zigzag/test_images.npy'))
mnist_c_l1=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/brightness/test_labels.npy'))
mnist_c_l2=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/canny_edges/test_labels.npy'))
mnist_c_l3=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/dotted_line/test_labels.npy'))
mnist_c_l4=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/fog/test_labels.npy'))
mnist_c_l5=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/glass_blur/test_labels.npy'))
mnist_c_l6=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/identity/test_labels.npy'))
mnist_c_l7=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/impulse_noise/test_labels.npy'))
mnist_c_l8=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/motion_blur/test_labels.npy'))
mnist_c_l9=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/rotate/test_labels.npy'))
mnist_c_l10=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/scale/test_labels.npy'))
mnist_c_l11=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/shear/test_labels.npy'))
mnist_c_l12=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/shot_noise/test_labels.npy'))
mnist_c_l13=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/spatter/test_labels.npy'))
mnist_c_l14=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/stripe/test_labels.npy'))
mnist_c_l15=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/translate/test_labels.npy'))
mnist_c_l16=torch.tensor(np.load('/content/drive/MyDrive/mnist_c/mnist_c/zigzag/test_labels.npy'))
mnistcx=torch.cat((mnist_c_d1,mnist_c_d2,mnist_c_d3,mnist_c_d4,mnist_c_d5,mnist_c_d6,mnist_c_d7,mnist_c_d8,mnist_c_d9,mnist_c_d10,mnist_c_d11,mnist_c_d12,mnist_c_d13,mnist_c_d14,mnist_c_d15,mnist_c_d16),dim=0).type(torch.FloatTensor)
mnistcy=torch.cat((mnist_c_l1,mnist_c_l2,mnist_c_l3,mnist_c_l4,mnist_c_l5,mnist_c_l6,mnist_c_l7,mnist_c_l8,mnist_c_l9,mnist_c_l10,mnist_c_l11,mnist_c_l12,mnist_c_l13,mnist_c_l14,mnist_c_l15,mnist_c_l16),dim=0).type(torch.FloatTensor)
mnistcx=mnistcx.permute(0,3,1,2)
print(mnistcx.shape)
print(mnistcy.shape)
mnistcdata =TensorDataset(mnistcx,mnistcy)
# for i in range(len(mnist_c_d1)):
#     mnistcdata.append([mnist_c_d1[i],mnist_c_l1[i]])
# for i in range(len(mnist_c_d2)):
#     mnistcdata[0].append(mnist_c_d2[i])
#     mnistcdata[1].append(mnist_c_l2[i])
# for i in range(len(mnist_c_d3)):
#     mnistcdata[0].append(mnist_c_d3[i])
#     mnistcdata[1].append(mnist_c_l3[i])
# for i in range(len(mnist_c_d4)):
#     mnistcdata[0].append(mnist_c_d4[i])
#     mnistcdata[1].append(mnist_c_l4[i])
# for i in range(len(mnist_c_d5)):
#     mnistcdata[0].append(mnist_c_d5[i])
#     mnistcdata[1].append(mnist_c_l5[i])
# for i in range(len(mnist_c_d6)):
#     mnistcdata[0].append(mnist_c_d6[i])
#     mnistcdata[1].append(mnist_c_l6[i])
# for i in range(len(mnist_c_d7)):
#     mnistcdata[0].append(mnist_c_d7[i])
#     mnistcdata[1].append(mnist_c_l7[i])
# for i in range(len(mnist_c_d8)):
#     mnistcdata[0].append(mnist_c_d8[i])
#     mnistcdata[1].append(mnist_c_l8[i])
# for i in range(len(mnist_c_d9)):
#     mnistcdata[0].append(mnist_c_d9[i])
#     mnistcdata[1].append(mnist_c_l9[i])
# for i in range(len(mnist_c_d10)):
#     mnistcdata[0].append(mnist_c_d10[i])
#     mnistcdata[1].append(mnist_c_l10[i])
# for i in range(len(mnist_c_d11)):
#     mnistcdata[0].append(mnist_c_d11[i])
#     mnistcdata[1].append(mnist_c_l11[i])
# for i in range(len(mnist_c_d12)):
#     mnistcdata[0].append(mnist_c_d12[i])
#     mnistcdata[1].append(mnist_c_l12[i])
# for i in range(len(mnist_c_d13)):
#     mnistcdata[0].append(mnist_c_d13[i])
#     mnistcdata[1].append(mnist_c_l13[i])
# for i in range(len(mnist_c_d14)):
#     mnistcdata[0].append(mnist_c_d14[i])
#     mnistcdata[1].append(mnist_c_l14[i])
# for i in range(len(mnist_c_d15)):
#     mnistcdata[0].append(mnist_c_d15[i])
#     mnistcdata[1].append(mnist_c_l15[i])
# for i in range(len(mnist_c_d16)):
#     mnistcdata[0].append(mnist_c_d16[i])
#     mnistcdata[1].append(mnist_c_l16[i])

mnistc_loader= torch.utils.data.DataLoader(mnistcdata,shuffle=False,batch_size=1000)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Extracting ./data/KMNIST/raw/train-labels-idx1-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/3041136 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/5120 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/KMNIST/raw

torch.Size([160000, 1, 28, 28])
torch.Size([160000])


# Data Augmentation techniques

1. Transformations
2. PixMix
3. Salt and Pepper


In [ ]:
#Augmentations (referred from PixMix github)
IMAGE_SIZE = 28
def int_parameter(level, maxval):
  return int(level * maxval / 10)


def float_parameter(level, maxval):
  return float(level) * maxval / 10.


def sample_level(n):
  return np.random.uniform(low=0.1, high=n)


def autocontrast(pil_img, _):
  return ImageOps.autocontrast(pil_img)


def equalize(pil_img, _):
  return ImageOps.equalize(pil_img)


def posterize(pil_img, level):
  level = int_parameter(sample_level(level), 4)
  return ImageOps.posterize(pil_img, 4 - level)


def rotate(pil_img, level):
  degrees = int_parameter(sample_level(level), 30)
  if np.random.uniform() > 0.5:
    degrees = -degrees
  return pil_img.rotate(degrees, resample=Image.BILINEAR)


def solarize(pil_img, level):
  level = int_parameter(sample_level(level), 256)
  return ImageOps.solarize(pil_img, 256 - level)


def shear_x(pil_img, level):
  level = float_parameter(sample_level(level), 0.3)
  if np.random.uniform() > 0.5:
    level = -level
  return pil_img.transform((IMAGE_SIZE, IMAGE_SIZE),
                           Image.AFFINE, (1, level, 0, 0, 1, 0),
                           resample=Image.BILINEAR)


def shear_y(pil_img, level):
  level = float_parameter(sample_level(level), 0.3)
  if np.random.uniform() > 0.5:
    level = -level
  return pil_img.transform((IMAGE_SIZE, IMAGE_SIZE),
                           Image.AFFINE, (1, 0, 0, level, 1, 0),
                           resample=Image.BILINEAR)


def translate_x(pil_img, level):
  level = int_parameter(sample_level(level), IMAGE_SIZE / 3)
  if np.random.random() > 0.5:
    level = -level
  return pil_img.transform((IMAGE_SIZE, IMAGE_SIZE),
                           Image.AFFINE, (1, 0, level, 0, 1, 0),
                           resample=Image.BILINEAR)


def translate_y(pil_img, level):
  level = int_parameter(sample_level(level), IMAGE_SIZE / 3)
  if np.random.random() > 0.5:
    level = -level
  return pil_img.transform((IMAGE_SIZE, IMAGE_SIZE),
                           Image.AFFINE, (1, 0, 0, 0, 1, level),
                           resample=Image.BILINEAR)
augmentations = [
    autocontrast, equalize, posterize, rotate, solarize, shear_x, shear_y,
    translate_x, translate_y
]

def get_ab(beta):
  if np.random.random() < 0.5:
    a = np.float32(np.random.beta(beta, 1))
    b = np.float32(np.random.beta(1, beta))
  else:
    a = 1 + np.float32(np.random.beta(1, beta))
    b = -np.float32(np.random.beta(1, beta))
  return a, b

def add(img1, img2, beta):
  a,b = get_ab(beta)
  img1, img2 = img1 * 2 - 1, img2 * 2 - 1
  out = a * img1 + b * img2
  return (out + 1) / 2

def multiply(img1, img2, beta):
  a,b = get_ab(beta)
  img1, img2 = img1 * 2, img2 * 2
  out = (img1 ** a) * (img2.clip(1e-37) ** b)
  return out / 2

mixings = [add, multiply]



In [ ]:
class PixMix(torch.utils.data.Dataset):
    def __init__(self, data, mix, preprocess):
        
        self.data = data
        self.mix = mix
        self.preprocess = preprocess
        
    
    def __getitem__(self, idx):
        
        x, y = self.data[idx]
        rnd_idx = np.random.choice(len(self.mix))
        mixing_pic, _ = self.mix[rnd_idx]
        return transforms.functional.rgb_to_grayscale(pixmix(x, mixing_pic, self.preprocess)), y

    
    def __len__(self):
        return len(self.data)

def pixmix(og,mix,preprocess):
    tensorize, normalize = preprocess['tensorize'], preprocess['normalize']
    if np.random.random() < 0.5:
      mixed = tensorize(augment(og))
    else:
      mixed = tensorize(og)
    
    for _ in range(np.random.randint(5)):
      
      if np.random.random() < 0.5:
        aug_image_copy = tensorize(augment(og))
      else:
        aug_image_copy = tensorize(mix)

      mixed_op = np.random.choice(mixings)
      mixed = mixed_op(mixed, aug_image_copy, 3)
      mixed = torch.clip(mixed, 0, 1)
      mixed.shape
      #plt.imshow(mixed.permute(1,2,0))
    return normalize(mixed)

def augment(image):
    op = np.random.choice(augmentations)
    return op(image.copy(), 3)

#to load images
class MixImages(torch.utils.data.Dataset):
    def __init__(self, transform):
        
        self.dir = '/content/drive/MyDrive/images/'
        self.transform=transform
        
    
    def __getitem__(self, idx):
        
        ind = '1 ('+str(idx+1)+').png'
        img=PIL.Image.open(self.dir+ind)
        return self.transform(img), 0

    
    def __len__(self):
        return 4690

#to load mnistc
'''class MNISTCData(torch.utils.data.Dataset):
    def __init__(self,path1,path2, transform):
        
        self.data = '/content/drive/MyDrive/images/'
        self.labels= 
        self.transform=transform
        
    
    def __getitem__(self, idx):
        
        
        return , self.labels[idx]

    
    def __len__(self):
        return len(self.labels)'''



class SaltPepper(torch.utils.data.Dataset):
    def __init__(self, data):
       
        self.data=data
    
    def __getitem__(self, idx):
        x,y=self.data[idx]
        if idx%2==0:
          salt_img = torch.tensor(random_noise(x, mode='salt', amount=0.075))
          return salt_img, y
        else:
          return x,y
    
    def __len__(self):
        return len(self.data)

class Augment(torch.utils.data.Dataset):
    def __init__(self, data):
       
        self.data=data
    
    def __getitem__(self, idx):
        x,y=self.data[idx]
        op = np.random.choice(augmentations)
        aug_img = op(x.copy(),3)
        return aug_img, y
    
    def __len__(self):
        return len(self.data)


#Anomaly Detection
##AUROC score
##Calculating anomaly detection scores
1. Max Softmax Equation
2. Cross-Entropy Anomaly equation

In [ ]:
#General Functions

concat = lambda x: np.concatenate(x, axis=0)
to_np = lambda x: x.data.cpu().numpy()

def get_ood_scores(loader, anomaly_score_calculator, model_net, use_penultimate = False):
    _score = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(loader):
            
            if torch.cuda.is_available():
                fraction = 1000
                
            else:
                fraction = 1000
            if batch_idx >= 50:
                break
            if torch.cuda.is_available():
                data = data.cuda()
            
            output = model_net(data)
            #print(output.shape)
            score = anomaly_score_calculator(output)
            _score.append(score)
            #print(_score)
    
    return concat(_score).copy()

def get_and_print_results(ood_loader, anomaly_score_calculator, model_net, use_penultimate):
    out_score = get_ood_scores(ood_loader, anomaly_score_calculator, model_net, use_penultimate = use_penultimate)
    auroc = get_auroc(out_score, in_score)
    print('AUROC: \t\t\t{:.2f}'.format(100 * auroc) + "%")
    return auroc

def print_all_results(anomaly_score_calculator, anomaly_score_name, model_net, train_loader, model_name = "default_model", use_penultimate = False):
    global in_score, all_anomaly_results
    in_score = get_ood_scores(train_loader, anomaly_score_calculator, model_net, use_penultimate)
    results = []

    print('Fashion MNIST')
    auroc = get_and_print_results(fmnist_loader, anomaly_score_calculator, model_net, use_penultimate)

    print('FakeData')
    auroc = get_and_print_results(fake_loader, anomaly_score_calculator, model_net, use_penultimate)

    # results.append(auroc)

    # if not model_name in all_anomaly_results:
    #     all_anomaly_results[model_name] = {}
    # all_anomaly_results[model_name][anomaly_score_name] = results

def get_auroc(_pos, _neg):
    ood_data=np.ones_like(_pos)
    in_data=np.zeros_like(_neg)
    final_labels= np.concatenate((ood_data,in_data),axis=0)
    final_scores= np.concatenate((_pos,_neg),axis=0)
    auroc_score= sk.roc_auc_score(final_labels,final_scores)
    print(auroc_score)


    return auroc_score

def max_softmax_anomaly_score(output):
    score_exp = torch.exp(output)
    #print(score_exp)
    sum= torch.sum(score_exp,dim=1)
    score_exp= torch.div(score_exp.t(),sum).t()
    max_score, inds = torch.max(score_exp,dim=1)
    max_score=-1*max_score
    
    return max_score.cpu().numpy()

def cross_entropy_anomaly_score(output):
   
    mean = torch.mean(output,dim=1)
    scores_exp_sum= torch.sum(torch.exp(output),dim=1)
    log_scores=torch.log(scores_exp_sum)
    final_score= mean-log_scores
    
    return final_score.cpu().numpy()

In [ ]:


class Net(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()  #Dropout
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        #Convolutional Layer/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) 
        #Convolutional Layer/Dropout/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        #Fully Connected Layer/Activation
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        #Fully Connected Layer/Activation
        x = self.fc2(x)
        #Softmax gets probabilities. 
        return F.log_softmax(x, dim=1)

#Model with batch norm.
class Net1(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()  #Dropout
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        #Convolutional Layer/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) 
        #Convolutional Layer/Dropout/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        #Fully Connected Layer/Activation
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        #Fully Connected Layer/Activation
        x = self.fc2(x)
        #Softmax gets probabilities. 
        return F.log_softmax(x, dim=1)


In [ ]:

def train(epoch,train_loader,model,optimizer):
    
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        #Variables in Pytorch are differenciable. 
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        output = model(data)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        loss = F.nll_loss(output, target)
        #dloss/dx for every Variable 
        loss.backward()
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically. 
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))
def train_OE(epoch,train_loader,model_OE,optimizer):
    model_OE.train()

    fmnist_loader.dataset.offset = np.random.randint(len(fmnist_loader.dataset))
    for in_set, out_set in zip(train_loader, fmnist_loader):
        data = torch.cat((in_set[0], out_set[0]), 0)
        target = in_set[1]
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        #Variables in Pytorch are differenciable. 
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        output = model(data)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        #loss= F.nll_loss(output[:len(in_set[0])], target)
        loss = F.cross_entropy(output[:len(in_set[0])], target)
        loss += 0.5 * -(output[len(in_set[0]):].mean(1) - torch.logsumexp(output[len(in_set[0]):], dim=1)).mean()
        #dloss/dx for every Variable 
        loss.backward()
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically. 
        '''if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))'''


def test(model,test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        target=target.type(torch.LongTensor).cuda()
        with torch.no_grad():
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False) # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))







In [ ]:
from numpy.core.arrayprint import printoptions

mixing_set = MixImages(transform=transforms.Compose(
      [transforms.Resize(36),
       transforms.RandomCrop(28)]))
pixmix_train = PixMix(train_data1,mixing_set,{'normalize': normalize, 'tensorize': to_tensor})
pixmix_loader = torch.utils.data.DataLoader(pixmix_train,
    batch_size=args['batch_size'], shuffle=True,num_workers=2,pin_memory=True, **kwargs)
sandp_train= SaltPepper(train_data)
sandp_loader = torch.utils.data.DataLoader(sandp_train,
    batch_size=args['batch_size'], shuffle=True, **kwargs)
aug_train= SaltPepper(train_data)
aug_loader = torch.utils.data.DataLoader(aug_train,
    batch_size=args['batch_size'], shuffle=True, **kwargs)

global model
global modelaug
global modelP
global modelP_OE
global modelS
global modelS_OE
global optimizer
global optimizerOE
global optimizerP
global optimizerPOE
global optimizerS
global optimizerSOE
model = Net()
modelaug = Net()
modelP = Net()
#modelP_OE = Net()
modelS = Net()
#modelS_OE = Net()



if args['cuda']:
    model.cuda()
    modelaug.cuda()
    modelP.cuda()
    #modelP_OE.cuda()
    modelS.cuda()
    #modelS_OE.cuda()

optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])
optimizeraug = optim.SGD(modelaug.parameters(), lr=args['lr'], momentum=args['momentum'])
optimizerP = optim.SGD(modelP.parameters(), lr=args['lr'], momentum=args['momentum'])
#optimizerPOE = optim.SGD(modelP_OE.parameters(), lr=args['lr'], momentum=args['momentum'])
optimizerS = optim.SGD(modelS.parameters(), lr=args['lr'], momentum=args['momentum'])
#optimizerSOE = optim.SGD(modelS_OE.parameters(), lr=args['lr'], momentum=args['momentum'])

# for epoch in range(1, args['epochs'] + 1):
#     train(epoch,aug_loader,modelaug,optimizer)

# print("Accuracy of AUG Model")
# test(modelaug,test_loader)
# print("Robustness on AUG Model")
# test(modelaug,mnistc_loader)
# #print("AUROC of Base Model")
# print_all_results(max_softmax_anomaly_score, "Max Softmax", modelaug,train_loader)
# print_all_results(cross_entropy_anomaly_score, "Cross Entropy", modelaug,train_loader)

# for epoch in range(1,6):   
#     train_OE(epoch,aug_loader,modelaug,optimizer)

# print("Accuracy of AUG Model with Outlier Exposure")
# test(modelaug,test_loader)
# print("Robustness on AUG Model with Outlier Exposure")
# test(modelaug,mnistc_loader)
# print("AUROC of Base AUG with OE")
# print_all_results(max_softmax_anomaly_score, "Max Softmax", modelaug,train_loader)
# print_all_results(cross_entropy_anomaly_score, "Cross Entropy", modelaug,train_loader)

for epoch in range(1, args['epochs'] + 1):
    train(epoch,train_loader,model,optimizer)

print("Accuracy of Base Model")
test(model,test_loader)
print("Robustness on Base Model")
test(model,mnistc_loader)
#print("AUROC of Base Model")
print_all_results(max_softmax_anomaly_score, "Max Softmax", model,train_loader)
print_all_results(cross_entropy_anomaly_score, "Cross Entropy", model,train_loader)

for epoch in range(1,6):   
    train_OE(epoch,train_loader,model,optimizer)

print("Accuracy of Base Model with Outlier Exposure")
test(model,test_loader)
print("Robustness on Base Model with Outlier Exposure")
test(model,mnistc_loader)
print("AUROC of Base Model with OE")
print_all_results(max_softmax_anomaly_score, "Max Softmax", model,train_loader)
print_all_results(cross_entropy_anomaly_score, "Cross Entropy", model,train_loader)

for epoch in range(1, args['epochs'] + 1):
    train(epoch,sandp_loader,modelS,optimizerS)

print("Accuracy of Model trained with Salt and Pepper")
test(modelS,test_loader)
print("Robustness on Model trained with Salt and Pepper")
test(modelS,mnistc_loader)
print("AUROC of Salt and Pepper Model")
print_all_results(max_softmax_anomaly_score, "Max Softmax", modelS,train_loader)
print_all_results(cross_entropy_anomaly_score, "Cross Entropy", modelS,train_loader)

for epoch in range(1,6): 
    train_OE(epoch,sandp_loader,modelS,optimizerS)

print("Accuracy of OE+Salt and Pepper")
test(modelS,test_loader)
print("Robustness on OE+Salt and Pepper")
test(modelS,mnistc_loader)
print("AUROC of Salt and Pepper Model with OE")
print_all_results(max_softmax_anomaly_score, "Max Softmax", modelS,train_loader)
print_all_results(cross_entropy_anomaly_score, "Cross Entropy", modelS,train_loader)

for epoch in range(1, args['epochs'] + 1):
    train(epoch,pixmix_loader,modelP,optimizerP)

print("Accuracy of Model with PixMix")
test(modelP,test_loader)
print("Robustness on Model trained with PixMix")
test(modelP,mnistc_loader)
print("AUROC of PixMix Model")
print_all_results(max_softmax_anomaly_score, "Max Softmax", modelP,train_loader)
print_all_results(cross_entropy_anomaly_score, "Cross Entropy", modelP,train_loader)

for epoch in range(1,6):
    train_OE(epoch,pixmix_loader,modelP,optimizerP)

print("Accuracy of Model with OE+PixMix")
test(modelP,test_loader)
print("Robustness on OE+PixMix")
test(modelP,mnistc_loader)
print("AUROC of PixMix Model with OE")
print_all_results(max_softmax_anomaly_score, "Max Softmax", modelP,train_loader)
print_all_results(cross_entropy_anomaly_score, "Cross Entropy", modelP,train_loader)




Train Epoch: 1 [0/50000 (0%)]	Loss: 2.325965
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.323879
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.324531
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.317811
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.327005
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.320103
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.321556
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.326053
Train Epoch: 2 [30000/50000 (60%)]	Loss: 2.327157
Train Epoch: 2 [40000/50000 (80%)]	Loss: 2.322355
Train Epoch: 3 [0/50000 (0%)]	Loss: 2.321033
Train Epoch: 3 [10000/50000 (20%)]	Loss: 2.322567
Train Epoch: 3 [20000/50000 (40%)]	Loss: 2.318416
Train Epoch: 3 [30000/50000 (60%)]	Loss: 2.328295
Train Epoch: 3 [40000/50000 (80%)]	Loss: 2.320589
Train Epoch: 4 [0/50000 (0%)]	Loss: 2.321573
Train Epoch: 4 [10000/50000 (20%)]	Loss: 2.317251
Train Epoch: 4 [20000/50000 (40%)]	Loss: 2.328310
Train Epoch: 4 [30000/50000 (60%)]	Loss: 2.322694
Train Epoch: 4 [40000/50000 (80%)]	Loss: 2.319806
Train Epoch: 5 [0/50

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 2.3173, Accuracy: 1355/10000 (14%)

Robustness on AUG Model

Test set: Average loss: 11.2864, Accuracy: 16133/160000 (10%)

Fashion MNIST
0.9605402003333334
AUROC: 			96.05%
FakeData
0.9504578749999999
AUROC: 			95.05%
Fashion MNIST
0.9968546738333333
AUROC: 			99.69%
FakeData
0.996516451
AUROC: 			99.65%


KeyboardInterrupt: ignored